In [1]:
import sys,os
sys.path.insert(0, os.path.abspath(".."))

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import os

: 

In [ ]:
target='formation_energy_per_site'
cfid_path = f'../results_2/2dmd/CatBoostRegressor/{target}/cfid/all_density/'
X_train = pd.read_csv(os.path.join(cfid_path, 'X_train.csv'),index_col=0)
X_test = pd.read_csv(os.path.join(cfid_path, 'X_test.csv'),index_col=0)
y_train = pd.read_csv(os.path.join(cfid_path, 'y_train.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(cfid_path, 'y_test.csv'),index_col=0)

In [3]:

sample_weight = pd.read_csv(os.path.join(cfid_path, 'sample_weight.csv'),index_col=0)

In [4]:
test_data = pd.merge(X_test, y_test, left_index=True, right_index=True)
train_data = pd.merge(X_train, y_train, left_index=True, right_index=True)

In [5]:
from pycaret.regression import *
s = RegressionExperiment()
s.setup(data=train_data, target = target,
         session_id = 42,
        train_size=0.8,fold=5, #feature_selection=True,
        n_jobs=-1)

,Description,Value
0,Session id,42
1,Target,formation_energy_per_site
2,Target type,Regression
3,Original data shape,"(11892, 1558)"
4,Transformed data shape,"(11892, 1558)"
5,Transformed train set shape,"(9513, 1558)"
6,Transformed test set shape,"(2379, 1558)"
7,Numeric features,1557
8,Preprocess,True
9,Imputation type,simple


In [6]:
best =s.compare_models(n_select=10, sort = "MAE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0478,0.0188,0.1364,0.9894,0.0411,0.0342,15.0620
rf,Random Forest Regressor,0.0529,0.0203,0.1422,0.9884,0.0457,0.0796,22.8500
catboost,CatBoost Regressor,0.0600,0.0187,0.1362,0.9894,0.0428,0.0645,40.7720
lightgbm,Light Gradient Boosting Machine,0.0619,0.0220,0.1480,0.9875,0.0464,0.0988,7.8240
dt,Decision Tree Regressor,0.0664,0.0377,0.1941,0.9785,0.0606,0.0770,1.4500
gbr,Gradient Boosting Regressor,0.0966,0.0315,0.1771,0.9821,0.0580,0.1509,15.5660
knn,K Neighbors Regressor,0.1203,0.0691,0.2628,0.9605,0.1064,0.3358,1.9820
ada,AdaBoost Regressor,0.3493,0.1776,0.4210,0.8986,0.1575,0.7372,11.6820
en,Elastic Net,0.7247,0.8573,0.9259,0.5111,0.3271,1.1691,1.9720
lr,Linear Regression,0.7380,0.9037,0.9472,0.4868,0.3351,1.0567,2.7680


In [7]:
from sklearn.metrics import mean_absolute_error
trained_models = []
weighted_mae_results = []
for model in best:
    finalize = s.finalize_model(model)
    trained_models.append(finalize)



In [8]:
weighted_mae_results=[]
for trained_model in trained_models:
    y_pred_test = trained_model.predict(X_test)
    y_pred_train = trained_model.predict(X_train)
    weighted_mae_test = mean_absolute_error(y_test,y_pred_test,sample_weight=sample_weight['weight'].loc[y_test.index])
    weighted_mae_train = mean_absolute_error(y_train,y_pred_train,sample_weight=sample_weight['weight'].loc[y_train.index])
    test_train_ratio = weighted_mae_test/weighted_mae_train
    weighted_mae_results.append({
        'Model': type(trained_model.steps[-1][-1]).__name__,
        'test-Weighted_MAE': weighted_mae_test,
        'train-Weighted_MAE': weighted_mae_train,
        'test_train_ratio': test_train_ratio
    })
    

In [9]:
results_df = pd.DataFrame(weighted_mae_results).sort_values(by='test-Weighted_MAE')
results_df.round(6)

,Model,test-Weighted_MAE,train-Weighted_MAE,test_train_ratio
0,ExtraTreesRegressor,0.120187,0.000822,146.242731
1,RandomForestRegressor,0.132186,0.051331,2.575159
2,CatBoostRegressor,0.135922,0.075812,1.792894
3,LGBMRegressor,0.144314,0.089592,1.610800
4,DecisionTreeRegressor,0.172183,0.000822,209.511804
5,GradientBoostingRegressor,0.188651,0.165043,1.143045
6,KNeighborsRegressor,0.231429,0.176012,1.314846
7,AdaBoostRegressor,0.351942,0.341124,1.031712
8,ElasticNet,0.533177,0.524281,1.016968
9,LinearRegression,0.633033,0.596887,1.060558
